## Segmenting and Clustering Neighborhoods in Toronto

#### Author: Olumide Olumayegun

In this notebook, I will explore, segment and cluster the neighborhoods in the city of Toronto. The Toronto neighborhoods data will be obtained by scraping a  Wikipedia page. The data will be wrangled, cleaned and read into a pandas dataframe so that it is in a structured format.

#### Download and import all the required libraries

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #Library to handle JSON files

import requests # library to handle requests

# Use either geopy or geocoder

# for geopy:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed geopy yet
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# We use geocoder in this notebook:
#!conda install -c conda-forge geocoder # uncomment this line if you haven't installed geocoder yet
import geocoder

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you dont have folium installed yet
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Step 1: Using pandas to scrape the data and transform the data into a pandas dataframe

The Toronto neighborhood data is available at the following Wikipedia page:  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
dfs = dfs[0]
dfs.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Drop all rows with a borough that is not assigned

In [4]:
# Setting the condition for dropping the rows
condition = dfs["Borough"]=="Not assigned"
# Dropping rows based on the condition
dfs.drop(dfs[condition].index, axis=0, inplace=True)
# Reset index
dfs = dfs.reset_index(drop=True)
dfs.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# Check for NaN
#dfs['Neighborhood'].isnull().values.any() # Check for NaN under Neighborhood column
#dfs['Neighborhood'].isnull().sum() # Count the NaN under Neighborhood column
#dfs.isnull().values.any() # Check for NaN in the entire DaraFrame
dfs.isnull().sum().sum() # Count the NaN in th entire dataframe

0

In [6]:
# Check for any not assigned Neighborhood
not_assigned_neighborhood = dfs[dfs["Neighborhood"] == "Not assigned"].shape[0]
not_assigned_neighborhood

0

Fortunately, there is no NaN cell or not assigned neighorhood in the dataframe

In [7]:
# Printing the number of rows in the dataframe
dfs.shape

(103, 3)

### Step 2: Getting the latitude and longitude coordinates of each neighborhood

Use the Geocoder Python package to get the latitude and longitude coordinates of each postal code.

Note: geocoder.google() method always return None, so I used geocoder.arcgis() method instead.

In [8]:
latitude = np.zeros((dfs.shape[0])) # Initialise latitude values with zeros
longitude = np.zeros((dfs.shape[0])) # Initialise longitude values with zeros

for i, postal_code in enumerate(dfs["Postal Code"]):
    lat_lng_coords = None # Initialialise variable to none
    
    while (lat_lng_coords is None): #Loop until you get the coordinates
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code)) # Note: goecoder.google does not work, use geocoder.arcgis instead
        lat_lng_coords = g.latlng
            
    latitude[i] = lat_lng_coords[0]
    longitude[i] = lat_lng_coords[1]

dfs["Latitude"] = latitude    # Add latitude column to the dataframe
dfs["Longitude"] = longitude  # Add longitude column to the dataframe
dfs.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667481,-79.528953
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


### Step 3: Exploring and clustering the neighborhoods in Toronto

Use geocoder library to get the latitude and longitude values of Toronto

In [9]:
address = 'Toronto, Ontario'
geolocator = geocoder.arcgis(address)
latitude_Toronto = geolocator.latlng[0]
longitude_Toronto = geolocator.latlng[1]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_Toronto, longitude_Toronto))
#print(latitude_Toronto, longitude_Toronto)

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


Create a map of Toronto with neighborhoods superimposed on top

In [10]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude_Toronto, longitude_Toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfs['Latitude'], dfs['Longitude'], dfs['Borough'], dfs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [11]:
# Check the number of postal codes each each borough
dfs_count = dfs.groupby('Borough')['Postal Code'].count()
dfs_count

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Postal Code, dtype: int64

There are ten boroughs in Toronto, four of which contain the word Toronto in their name.
Hence for the purpose of analysis, I will segment and cluster only the neighborhoods in these four boroughs

In [12]:
# Slicing the original dataframe and create a new dataframe of neighborhoods with Toronto in Borough name
condition2 = []
for bor in dfs["Borough"]: 
    condition2.append("Toronto" in bor)

dfs_Toronto = dfs[condition2].reset_index(drop=True)
dfs_Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349
5,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855
6,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653
7,M6G,Downtown Toronto,Christie,43.668602,-79.420387
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664


Central Toronto is expected to be at the centre of the new Toronto neighborhood.
So let's get the geographical coordinates of Central Toronto.

In [13]:
address2 = 'Central Toronto, Ontario'
geolocator = geocoder.arcgis(address2)
latitude_central = geolocator.latlng[0]
longitude_central = geolocator.latlng[1]
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude_central, longitude_central))
#print(latitude_Toronto, longitude_Toronto)

The geograpical coordinate of Central Toronto are 43.60972747387443, -79.49284428510224.


Let's us visulaise the neighborhoods in the four Toronto boroughs

In [14]:
# create map of Toronto boroughs using latitude and longitude values
map_newToronto = folium.Map(location=[latitude_central, longitude_central], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfs_Toronto['Latitude'], dfs_Toronto['Longitude'], dfs_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newToronto)  
    
map_newToronto

Define Foursquare credential and version

In [15]:
CLIENT_ID = 'I0HY4U4VDCPCPW0T3R0ZYMTKB2M2NH0LFUYP2XQ5XY0B02HF' # your Foursquare ID
CLIENT_SECRET = 'DNFTBSQHPO0EUG3GMPVSRFKE3EPWYWXJCFOH4UFCOGZMWXK5' # your Foursquare Secret
VERSION = '20200505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I0HY4U4VDCPCPW0T3R0ZYMTKB2M2NH0LFUYP2XQ5XY0B02HF
CLIENT_SECRET:DNFTBSQHPO0EUG3GMPVSRFKE3EPWYWXJCFOH4UFCOGZMWXK5


Let explore the first neighborhood in the dataframe

In [16]:
dfs_Toronto.loc[0,'Neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values

In [17]:
neighborhood_latitude = dfs_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfs_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfs_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65096410900003, -79.35304116399999.


Now, let's get the top 100 venues that are in Regent Park and Harbourfront within a radius of 500 meters.

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I0HY4U4VDCPCPW0T3R0ZYMTKB2M2NH0LFUYP2XQ5XY0B02HF&client_secret=DNFTBSQHPO0EUG3GMPVSRFKE3EPWYWXJCFOH4UFCOGZMWXK5&v=20200505&ll=43.65096410900003,-79.35304116399999&radius=500&limit=100'

Send the GET request and examine results

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecc242e9388d7001b78e228'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 43.65546411350003,
    'lng': -79.34683350482591},
   'sw': {'lat': 43.646464104500026, 'lng': -79.35924882317407}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57e0745a498ea809dbf75f68',
       'name': 'Souk Tabule',
       'location': {'address': '494 Front St E',
        'crossStreet': 'at Bayview',
        'lat': 43.65375556880743,
        'lng': -79.35439006096168,
        'labeledLatLngs

We know that all the information is in the *items* key

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now clean the json and structure it into pandas dataframe

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
2,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
3,BATLgrounds,Athletics & Sports,43.647088,-79.351306
4,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843


In [22]:
# Number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


#### Create a function to repeat the same process to all the neighborhood in Toronto area

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [24]:
toronto_venues = getNearbyVenues(names=dfs_Toronto['Neighborhood'],
                                   latitudes=dfs_Toronto['Latitude'],
                                   longitudes=dfs_Toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [25]:
# Let's check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1600, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [26]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,64,64,64,64,64,64
"Brockton, Parkdale Village, Exhibition Place",43,43,43,43,43,43
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",64,64,64,64,64,64
Central Bay Street,56,56,56,56,56,56


In [27]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


#### Analysing each neighborhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe and insert as the first column
#toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.insert(loc=0, column='Neighbourhood', value=toronto_venues['Neighborhood'])

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
#Examine the new dataframe
toronto_onehot.shape

(1600, 229)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.015625,0.031250,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.031250,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00,0.00,0.015625,0.046875,0.093750,0.000000,0.000000,0.000000,0.00,0.000000,0.015625,0.000000,0.015625,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.015625,0.0,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.0000

In [40]:
# Confirm the new size
toronto_grouped.shape

(39, 229)

Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2    Restaurant  0.03
3      Beer Bar  0.03
4          Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.07
2  Thrift / Vintage Store  0.05
3               Gift Shop  0.05
4                   Diner  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2  Japanese Restaurant  0.03
3                 Café  0.03
4     Asian Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0               Café  0.06
1        Coffee Shop  0.06
2         Restaurant  0.05
3               Park  0.05
4  French Restaurant  0.05


----Central Bay Street----
                 venue  

Let's put that into pandas dataframe

In [42]:
# First, let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Lounge,Beer Bar,Seafood Restaurant,Restaurant,Hotel,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Diner,Gift Shop,Thrift / Vintage Store,Accessories Store,Restaurant,Boutique,Italian Restaurant,Brewery
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Asian Restaurant,Steakhouse,Taco Place,Italian Restaurant,Tea Room
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Park,Restaurant,French Restaurant,Gym,Gym / Fitness Center,Speakeasy,Italian Restaurant,Lounge
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Clothing Store,Sandwich Place,Bubble Tea Shop,Plaza,Restaurant,Park,Business Service


#### Clustering neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfs_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Café,Athletics & Sports,Distribution Center,Music Venue,Seafood Restaurant,Bank,Bakery,Tech Startup,Thai Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Italian Restaurant,Fried Chicken Joint,Bookstore,Creperie,Burrito Place,Yoga Studio
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Hotel,Café,Cosmetics Shop,Bar
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Cosmetics Shop,American Restaurant,Gastropub,Theater,Creperie,Clothing Store
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Trail,Pub,Church,Neighborhood,Health Food Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop


In [46]:
# Visualizing the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude_central, longitude_central], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining clusters

Examine each cluster and determine the descriminating venue categories that distinguish each cluster

Cluster 1

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Pub,Café,Athletics & Sports,Distribution Center,Music Venue,Seafood Restaurant,Bank,Bakery,Tech Startup,Thai Restaurant
1,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Italian Restaurant,Fried Chicken Joint,Bookstore,Creperie,Burrito Place,Yoga Studio
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Restaurant,Italian Restaurant,Hotel,Café,Cosmetics Shop,Bar
3,Downtown Toronto,0,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Cosmetics Shop,American Restaurant,Gastropub,Theater,Creperie,Clothing Store
4,East Toronto,0,Trail,Pub,Church,Neighborhood,Health Food Store,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Lounge,Beer Bar,Seafood Restaurant,Restaurant,Hotel,Café
6,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Clothing Store,Sandwich Place,Bubble Tea Shop,Plaza,Restaurant,Park,Business Service
8,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Sushi Restaurant,Gym,Salad Place
10,Downtown Toronto,0,Theme Park,Harbor / Marina,Park,Farm,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
11,West Toronto,0,Bar,Coffee Shop,Cocktail Bar,Asian Restaurant,Restaurant,Wine Bar,Yoga Studio,Mac & Cheese Joint,Italian Restaurant,Japanese Restaurant


Cluster 2

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Café,Grocery Store,Coffee Shop,Baby Store,Candy Store,Athletics & Sports,Playground,Park,Farmers Market,Fast Food Restaurant
9,West Toronto,1,Park,Pharmacy,Pool,Café,Furniture / Home Store,Bank,Bakery,Bus Line,Gym,Grocery Store
12,East Toronto,1,Bus Line,Park,Business Service,Grocery Store,Discount Store,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
23,Central Toronto,1,Playground,Gym Pool,Park,Garden,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
33,Downtown Toronto,1,Playground,Park,Grocery Store,Candy Store,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 3

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Bus Line,Swim School,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Health & Beauty Service,Ice Cream Shop,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,4,Park,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant
